In [2]:
import regex as re


def create_ngrams(words: list[str], n: int) -> list[str]:
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = words[i:i+n]
        ngram_string = ' '.join(ngram)
        ngrams.append(ngram_string)
    
    return ngrams

def parse_sentence_to_ngrams(sentence: str) -> list[str]:
    words = re.sub(r'[^\p{L}\s]', '', sentence).split()
    words_count = len(words)

    if words_count > 20:
        return create_ngrams(words, 5)
    if words_count > 14:
        return create_ngrams(words, 4)
    if words_count > 8:
        return create_ngrams(words, 3)
    if words_count > 2:
        return create_ngrams(words, 2)
    else:
        return words

In [7]:
def parse_sentence_to_ngrams_2(sentence: str, n: int) -> list[str]:
    words = re.sub(r'[^\p{L}\s]', '', sentence).split()

    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = words[i:i+n]
        ngram_string = ' '.join(ngram)
        ngrams.append(ngram_string)
    
    return ngrams

In [13]:
import subprocess

def save_texts_with_ngrams_in_file(ngrams: list[str], source_filename: str, temp_filename: str):
    try:
        args = ['grep', '--fixed-strings', '--ignore-case', '--line-number']
        for ngram in ngrams:
            args.extend(['-e', ngram])
        args.append(source_filename)

        with open(temp_filename, 'w', encoding='utf-8') as f:
            subprocess.run(
                args=args,
                stdout=f,
                stderr=subprocess.PIPE,
                text=True,
            )
 

    except subprocess.CalledProcessError as e:
        # Obsługa błędów, gdy grep zwróci niezerowy kod wyjścia
        print(f"Błąd: Komenda zakończyła się niepowodzeniem: {e}")
        print(f"Błędy: {e}")

    except FileNotFoundError as e:
        # Obsługa przypadku, gdy plik lub komenda nie zostanie znaleziona
        print(f"Nie znaleziono pliku lub komendy: {e}")

    except Exception as e:
        # Ogólna obsługa innych błędów
        print(f"Nieoczekiwany błąd: {e}")


In [2]:
# sentence = ""

# n = 1
# line_count = 
# while 
# parse_sentence_to_ngrams_2(sentence, )

from rapidfuzz import process, fuzz

with open("./data/train.pl.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()

    sentence = "uwzględniając wspólny wniosek Komisji Europejskiej i Wysokiego Przedstawiciela Unii do Spraw Zagranicznych i Polityki Bezpieczeństwa"
    res = process.extract(sentence, lines, scorer=fuzz.WRatio, limit=2)
    print(res)


[('uwzględniając wspólny wniosek Komisji Europejskiej i Wysokiego Przedstawiciela Unii do Spraw Zagranicznych i Polityki Bezpieczeństwa ,\n', 98.87640449438202, 147617), ('uwzględniając wspólny wniosek Komisji Europejskiej i Wysokiego Przedstawiciela Unii do Spraw Zagranicznych i Polityki Bezpieczeństwa ,\n', 98.87640449438202, 1605432)]


In [3]:
def read_line_n(filename: str, n: int):
    with open(filename, 'r') as f:
        content = subprocess.run(args=['sed', '-n', f'{n}p', filename], capture_output=True, text=True)
        return content.stdout

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


def create_model(
        src_filename: str,
        k_neighbors = 5, 
        n_grams = 3
    ):
    with open(src_filename, 'r', encoding='utf-8') as f:
        text_list = []
        file_lines = f.readlines()
        for line in file_lines:
            text_list.append(line.strip())

        vectorizer = TfidfVectorizer(min_df=1, analyzer='word')
        tfidf_matrix = vectorizer.fit_transform(text_list)

        nbrs_model = NearestNeighbors(
            n_neighbors=k_neighbors, 
            n_jobs=-2, 
            metric="cosine"
        ).fit(tfidf_matrix)

        return nbrs_model, vectorizer

In [5]:
model, vectorizer = create_model('./data/train.en.txt')

In [10]:
from typing import TypedDict, List


class SentenceMatch(TypedDict):
    text: str
    translation: str
    confidence: float

def predict_fuzzy_matches(
        sentence: str,
        src_filename: str, 
        translation_filename: str, 
        ) -> List[SentenceMatch]:
    with open(src_filename, 'r', encoding='utf-8') as f:
        text_list = []
        file_lines = f.readlines()
        for line in file_lines:
            text_list.append(line.strip())


        tfidf_sentence = vectorizer.transform([sentence])
        distances, positions = model.kneighbors(tfidf_sentence)

        confidences = [round(1 - dist, 2) for dist in distances[0]]
        result = []
        for i, position in enumerate(positions[0]):
            text = text_list[position]
            confidence = confidences[i]
            translation = read_line_n(translation_filename, position + 1)

            result.append(SentenceMatch({
                'text': text,
                'translation': translation,
                'confidence': confidence
            }))
        
        return result


In [22]:
examples = predict_fuzzy_matches("A number of amendments to the international conventions and", './data/train.en.txt', './data/train.pl.txt')

In [23]:
print(examples)

[{'text': 'apply , for the purpose of this Directive , amendments made to the International Conventions ;', 'translation': 'stosowania do celów niniejszej dyrektywy zmian dokonanych w Międzynarodowych Konwencjach ;\n', 'confidence': 0.7}, {'text': 'CONVENTIONS', 'translation': 'W stosownych przypadkach na podstawie krajowych ogólnie przyjętych zasad rachunkowości opartych na BAD zgłasza się » Redukcje wartości w odniesieniu do pozycji przeznaczonych do obrotu wycenianych według wartości godziwej « .\n', 'confidence': 0.64}, {'text': 'CONVENTIONS', 'translation': 'Zmniejszenie zobowiązań\n', 'confidence': 0.64}, {'text': 'Conventions', 'translation': 'Konwencje\n', 'confidence': 0.64}, {'text': 'Conventions', 'translation': 'Zmiany w odpisach aktualizujących z tytułu strat kredytowych i utraty wartości instrumentów kapitałowych na podstawie krajowych ogólnie przyjętych zasad rachunkowości opartych na BAD ( 12.0 )\n', 'confidence': 0.64}]


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from typing import TypedDict, List

class SentenceMatch(TypedDict):
    text: str
    translation: str
    confidence: float

def find_fuzzy_matches(
        sentence: str, 
        temp_filename: str , 
        translation_filename: str, 
        k_neighbors = 5, 
        n_grams = 3) -> List[SentenceMatch]:
    with open(temp_filename, 'r', encoding='utf-8') as f:
        line_number_list = []
        text_list = []

        file_lines = f.readlines()
        for line in file_lines:
            line_number, text = line.strip().split(':', 1)
            line_number_list.append(line_number)
            text_list.append(text)

        def analyzer(string: str, n = n_grams):
            string = re.sub(r'[^\p{L}\s]', r'', string)
            ngrams = zip(*[string[i:] for i in range(n)])
            return ["".join(ngram) for ngram in ngrams]

        vectorizer = TfidfVectorizer(min_df=1, analyzer=analyzer)
        tfidf_matrix = vectorizer.fit_transform(text_list)

        nbrs_model = NearestNeighbors(
            n_neighbors=k_neighbors, 
            n_jobs=-2, 
            metric="cosine"
        ).fit(tfidf_matrix)

        tfidf_sentence = vectorizer.transform([sentence])
        distances, positions = nbrs_model.kneighbors(tfidf_sentence)

        confidences = [round(1 - dist, 2) for dist in distances[0]]
        result = []
        for i, position in enumerate(positions[0]):
            text = text_list[position]
            line_number = int(line_number_list[position])
            confidence = confidences[i]
            translation = read_line_n(translation_filename, line_number)

            result.append(SentenceMatch({
                'text': text,
                'translation': translation,
                'confidence': confidence
            }))
        
        return result


In [5]:
import regex as re

def analyzer(string: str, n: int):
    string = re.sub(r'[^\p{L}\s]', r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return ["".join(ngram) for ngram in ngrams]

analyzer("hello my name is", 2)

['he',
 'el',
 'll',
 'lo',
 'o ',
 ' m',
 'my',
 'y ',
 ' n',
 'na',
 'am',
 'me',
 'e ',
 ' i',
 'is']

In [11]:
find_fuzzy_matches("wiadomości publicznej niedyskryminujące procedury w zakresie zawieszenia uczestnictwa ", './data/temp2.txt', './data/train.en.txt', 5, 3)

[{'text': 'Operator SIPS ustala i podaje do wiadomości publicznej niedyskryminujące procedury w zakresie zawieszenia uczestnictwa i uporządkowanego wypowiedzenia uczestnictwa w przypadku niespełniania przez danego uczestnika kryteriów dostępu i uczestnictwa .',
  'translation': 'It shall review the procedures at least annually .',
  'confidence': 0.68},
 {'text': 'podawanie do wiadomości publicznej :',
  'translation': 'to make publicly available the following :',
  'confidence': 0.43},
 {'text': 'dana instytucja zdrowia publicznego sporządza podawane do wiadomości publicznej oświadczenie zawierające :',
  'translation': 'the health institution draws up a declaration which it shall make publicly available , including :',
  'confidence': 0.38},
 {'text': 'Streszczenie sprawozdania rocznego podaje się do wiadomości publicznej . ” ;',
  'translation': 'internal and external audit findings and the follow-up to the audit recommendations and to the discharge recommendation ;',
  'confidence'

In [9]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"


def make_openai_api_call(context: str, prompt: str):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            { "role": "system", "content": context },
            { "role": "user", "content": prompt }
        ]
    )
    return response.choices[0].message.content

In [10]:
def check_file_length(filename: str):
    with open(file=filename, encoding='utf-8', mode='r') as f:
        lines = f.readlines()
        return len(lines)


In [ ]:
from typing import Dict, Literal


Language = Literal["polski","angielski"]
TEMP_FILENAME = "./data/temp.txt"
context = "Jesteś pomocnym bilingwalnym tłumaczem specjalizującym się w tłumaczeniach pomiędzy językiem polskim, a angielskim. Jako wynik zwracasz samo tłumaczenie."

dataset_filenames: Dict[Language, str] = {
    "angielski": "./data/train.en.txt",
    "polski" : "./data/train.pl.txt"
}

def translate(text: str, source_language: Language, target_language: Language, few_shots: int):
    if few_shots == 0:
        prompt = f"Przetłumacz z języka {source_language}ego na język {target_language}.\n" + \
                 f"{source_language}: {text}\n" + \
                 f"{target_language}:"
        return make_openai_api_call(context, prompt)
    else:
        ngrams = parse_sentence_to_ngrams(text)

        save_texts_with_ngrams_in_file(
            ngrams=ngrams, 
            source_filename=dataset_filenames[source_language], 
            temp_filename=TEMP_FILENAME
        )

        # Return None if number of results in temp file is not enough 
        file_len = check_file_length(TEMP_FILENAME)
        if file_len < few_shots:
            return None

        matches = find_fuzzy_matches(
            sentence=text, 
            temp_filename=TEMP_FILENAME, 
            translation_filename=dataset_filenames[target_language], 
            k_neighbors=few_shots, 
            n_grams=3
        )

        def create_shot(match: str):
            return f"{source_language}: {match.get('text')}\n" + \
                   f"{target_language}: {match.get('translation')}"

        prompt = f"Przetłumacz zdania z języka {source_language}ego" + \
                 f"na język {target_language}, biorąc pod uwagę " + \
                 f"przykłady tłumaczeń zdań podobnych.\n" + \
                 "\n".join([create_shot(m) for m in matches]) + \
                 f"\n{source_language}: {text}" + \
                 f"\n{target_language}: "
        return make_openai_api_call(context, prompt)

In [7]:
from typing import Dict, Literal


Language = Literal["polski","angielski"]
TEMP_FILENAME = "./data/temp.txt"
context = "Jesteś pomocnym bilingwalnym tłumaczem specjalizującym się w tłumaczeniach pomiędzy językiem polskim, a angielskim. Jako wynik zwracasz samo tłumaczenie."

dataset_filenames: Dict[Language, str] = {
    "angielski": "./data/train.en.txt",
    "polski" : "./data/train.pl.txt"
}

def translate(text: str, source_language: Language, target_language: Language, few_shots: int):
    if few_shots == 0:
        prompt = f"Przetłumacz z języka {source_language}ego na język {target_language}.\n" + \
                 f"{source_language}: {text}\n" + \
                 f"{target_language}:"
        return make_openai_api_call(context, prompt)
    else:
        ngrams = parse_sentence_to_ngrams(text)

        save_texts_with_ngrams_in_file(
            ngrams=ngrams, 
            source_filename=dataset_filenames[source_language], 
            temp_filename=TEMP_FILENAME
        )

        # Return None if number of results in temp file is not enough 
        file_len = check_file_length(TEMP_FILENAME)
        if file_len < few_shots:
            return None

        matches = find_fuzzy_matches(
            sentence=text, 
            temp_filename=TEMP_FILENAME, 
            translation_filename=dataset_filenames[target_language], 
            k_neighbors=few_shots, 
            n_grams=3
        )

        def create_shot(match: str):
            return f"{source_language}: {match.get('text')}\n" + \
                   f"{target_language}: {match.get('translation')}"

        prompt = f"Przetłumacz zdania z języka {source_language}ego" + \
                 f"na język {target_language}, biorąc pod uwagę " + \
                 f"przykłady tłumaczeń zdań podobnych.\n" + \
                 "\n".join([create_shot(m) for m in matches]) + \
                 f"\n{source_language}: {text}" + \
                 f"\n{target_language}: "
        return make_openai_api_call(context, prompt)

In [8]:
from typing import Dict, Literal


Language = Literal["polski","angielski"]
TEMP_FILENAME = "./data/temp.txt"
context = "Jesteś pomocnym bilingwalnym tłumaczem specjalizującym się w tłumaczeniach pomiędzy językiem polskim, a angielskim. Jako wynik zwracasz samo tłumaczenie."

dataset_filenames: Dict[Language, str] = {
    "angielski": "./data/train.en.txt",
    "polski" : "./data/train.pl.txt"
}

def translate_batch(text: str, source_language: Language, target_language: Language, n_shots: list[int]):
    translations = []
    
    ngrams = parse_sentence_to_ngrams(text)
    save_texts_with_ngrams_in_file(
        ngrams=ngrams, 
        source_filename=dataset_filenames[source_language], 
        temp_filename=TEMP_FILENAME
    )

    file_len = check_file_length(TEMP_FILENAME)
    matches = []
    if file_len != 0:
        matches = find_fuzzy_matches(
            sentence=text, 
            temp_filename=TEMP_FILENAME, 
            translation_filename=dataset_filenames[target_language], 
            k_neighbors=min([max(n_shots), file_len]), 
            n_grams=3
        )
    
    for n in n_shots:
        if n_shots == 0:
            prompt = f"Przetłumacz z języka {source_language}ego na język {target_language}.\n" + \
                    f"{source_language}: {text}\n" + \
                    f"{target_language}:"
            t = make_openai_api_call(context, prompt)
            translations.append(t)
        else:
            if len(matches) < n:
                translations.append('None')
                continue
            
            def create_shot(match: str):
                return f"{source_language}: {match.get('text')}\n" + \
                    f"{target_language}: {match.get('translation')}"

            prompt = f"Przetłumacz zdania z języka {source_language}ego" + \
                    f"na język {target_language}, biorąc pod uwagę " + \
                    f"przykłady tłumaczeń zdań podobnych.\n" + \
                    "\n".join([create_shot(m) for m in matches]) + \
                    f"\n{source_language}: {text}" + \
                    f"\n{target_language}: "
            t = make_openai_api_call(context, prompt)
            translations.append(t)
    return translations

In [ ]:
sentence = "Załącznik III , B.1"


translation = translate(sentence,  'polski', 'angielski', 10)
print(translation)


Annex III , B.1


In [9]:
from tqdm import tqdm
import os

val_en = './data/val.en.txt'
val_pl = './data/val.pl.txt'
val_res_zero = './data/10/00.txt'
val_res_two = './data/10/02.txt'
val_res_five = './data/10/05.txt'
# val_res_ten = './data/new/10.txt'

 
def ngram_translations(source_language: Language, target_langauge: Language, ngrams: list[str]):
    source_file = val_en if source_language == 'angielski' else val_pl

    with open(source_file, "r") as file,\
         open(val_res_zero, 'a') as file_res_zero,\
         open(val_res_two, 'a') as file_res_two,\
         open(val_res_five, 'a') as file_res_five:
        #  open(val_res_ten, 'a') as file_res_ten:
        for i, sentence in enumerate(tqdm(file)):
            os.remove('data/temp.txt')
            if i < 241:
                continue
            if i > 300:
                break

            sentence = sentence.strip()
            translations = translate_batch(sentence, source_language, target_langauge, ngrams)
            print(i, translations)

            file_res_zero.write(str(translations[0]).strip() + '\n')
            file_res_two.write(str(translations[1]).strip() + '\n')
            file_res_five.write(str(translations[2]).strip() + '\n')
            # file_res_ten.write(str(translations[3]).strip() + '\n')

In [7]:
ngram_translations('polski', 'angielski', [0, 2, 5])

NameError: name 'ngram_translations' is not defined

In [ ]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from typing import TypedDict, List

class SentenceMatch(TypedDict):
    text: str
    translation: str
    confidence: float

def find_fuzzy_matches(
        sentence: str, 
        temp_filename: str , 
        translation_filename: str, 
        k_neighbors = 5
    ) -> List[SentenceMatch]:
    with open(temp_filename, 'r', encoding='utf-8') as f:
        file_lines = f.readlines()

        vectorizer = TfidfVectorizer(min_df=1, analyzer='word')
        tfidf_matrix = vectorizer.fit_transform(file_lines)

        nbrs_model = NearestNeighbors(
            n_neighbors=k_neighbors, 
            n_jobs=-1, 
            metric="cosine"
        ).fit(tfidf_matrix)

        tfidf_sentence = vectorizer.transform([sentence])
        distances, positions = nbrs_model.kneighbors(tfidf_sentence)

        confidences = [round(1 - dist, 2) for dist in distances[0]]
        result = []
        for i, position in enumerate(positions[0]):
            text = file_lines[position]
            translation = read_line_n(translation_filename, position + 1)
            confidence = confidences[i]
            
            result.append(SentenceMatch({
                "position": position,
                'text': text,
                'translation': translation,
                'confidence': confidence
            }))
        
        return result


In [6]:

sentence = 'Wnioskodawca twierdził , że zakres produktu powinien zostać wyjaśniony w odniesieniu do zamknięć włazów z żeliwa sferoidalnego , a w szczególności należy wyjaśnić , czy ten rodzaj zamknięć włazów powinien mieścić się w zakresie definicji produktu objętego postępowaniem .'
find_fuzzy_matches(sentence, './data/train.pl.txt', './data/train.en.txt', 10)

: 

In [1]:
from rapidfuzz import process, fuzz, utils

with open("./data/train.pl.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()

    sentence = 'Wnioskodawca twierdził , że zakres produktu powinien zostać wyjaśniony w odniesieniu do zamknięć włazów z żeliwa sferoidalnego , a w szczególności należy wyjaśnić , czy ten rodzaj zamknięć włazów powinien mieścić się w zakresie definicji produktu objętego postępowaniem .'
    res = process.extract(sentence, lines, scorer=fuzz.QRatio, limit=10, processor=utils.default_process)
    
    for i in res:
        print(i)


('Z treści pierwotnego zawiadomienia o wszczęciu nie wynika , że zamknięcia włazów wykonane z żeliwa sferoidalnego zostały wyraźnie lub domyślnie wyłączone z definicji produktu , którego dotyczy postępowanie .\n', 54.43037974683544, 1884430)
('Wnioskodawca twierdził również , że spółka dominująca nie powinna była zostać objęta zakresem dochodzenia dotyczącego MET , ponieważ nie jest ani producentem , ani eksporterem produktu objętego postępowaniem .\n', 54.201680672268914, 477952)
('Wnioskodawca twierdził , że gatunek do produkcji taśm powinien zostać uznany za niepodlegający zakresowi dochodzenia pierwotnego i , co za tym idzie , nie powinien być objęty wspomnianymi powyżej środkami .\n', 53.276955602537, 110468)
('Wnioskodawca twierdził , że wszelkie dostosowanie ceny gazu , jaką płaci on na rosyjskim rynku krajowym , będzie nieuzasadnione , ponieważ zapisy księgowe przedsiębiorstwa w pełni odzwierciedlają koszty związane z produkcją produktu objętego postępowaniem w Rosji .\n', 53.0